In [26]:
import pandas as pd
import numpy as np

# Cargar el archivo parquet con pandas
df = pd.read_parquet('/home/nicolas/despliegue_analytica/files_parquet/df_joined_cleaned_no_nulls.parquet')

# Asegurarse de que las fechas están en formato datetime
df['order_purchase_timestamp_dt'] = pd.to_datetime(df['order_purchase_timestamp'])

# Extraer el mes
df['month'] = df['order_purchase_timestamp_dt'].dt.month

# Función para asignar la temporada
def assign_season(month):
    if month in [12, 1, 2]:
        return 'Invierno'
    elif month in [3, 4, 5]:
        return 'Primavera'
    elif month in [6, 7, 8]:
        return 'Verano'
    else:
        return 'Otoño'

# Aplicar la función para crear la columna de temporada
df['season'] = df['month'].apply(assign_season)

# Calcular el valor total de ventas
df['total_value'] = df['payment_value']

# Agrupar por temporada y categoría, y sumar las ventas
seasonal_sales = df.groupby(['season', 'product_category_name'])['total_value'].sum().reset_index()

# Ordenar por temporada y valor de ventas en orden descendente
seasonal_sales = seasonal_sales.sort_values(['season', 'total_value'], ascending=[True, False])

# Eliminar duplicados para obtener el producto de mayor valor de ventas por temporada
top_products_per_season = seasonal_sales.drop_duplicates(subset=['season'], keep='first')

# Mostrar resultados
print(top_products_per_season)

# 1. Visualizar las dimensiones del DataFrame
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

# 2. Obtener información general sobre el DataFrame
print(df.info())  # Muestra información general del DataFrame

# 3. Describir las estadísticas descriptivas de las columnas numéricas
print(df.describe())  # Esto devuelve un DataFrame con estadísticas descriptivas

# 4. Ver las primeras filas del DataFrame
print(df.head())  # Muestra las primeras filas del DataFrame


        season   product_category_name  total_value
42    Invierno  informatica_acessorios    441329.14
82       Otoño         cama_mesa_banho    337702.86
153  Primavera         cama_mesa_banho    496289.75
223     Verano            beleza_saude    610064.95
Número de filas: 116583
Número de columnas: 35
<class 'pandas.core.frame.DataFrame'>
Index: 116583 entries, 0 to 119142
Data columns (total 35 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       116583 non-null  object        
 1   customer_id                    116583 non-null  object        
 2   order_status                   116583 non-null  object        
 3   order_purchase_timestamp       116583 non-null  datetime64[ns]
 4   order_approved_at              116583 non-null  datetime64[ns]
 5   order_estimated_delivery_date  116583 non-null  datetime64[ns]
 6   order_item_id                  116583 

In [ ]:
import pandas as pd

# Seleccionar las columnas relevantes
variables_relevantes = ['season', 'price', 'freight_value', 'payment_type', 'product_category_name', 'customer_state', 'month', 'total_value']

df_model = df[variables_relevantes].copy()

# Crear variables dummies para las variables categóricas
df_model = pd.get_dummies(df_model, columns=['season', 'payment_type', 'product_category_name', 'customer_state'], drop_first=True)

# Verificar el resultado
print(df_model.head(30))


      price  freight_value  month  total_value  season_Otoño  \
0     29.99           8.72     10        18.12          True   
1     29.99           8.72     10         2.00          True   
2     29.99           8.72     10        18.59          True   
3    118.70          22.76      7       141.46         False   
4    159.90          19.22      8       179.12         False   
5     45.00          27.20     11        72.20          True   
6     19.90           8.72      2        28.62         False   
7    147.90          27.36      7       175.26         False   
9     59.99          15.17      5        75.16         False   
10    19.90          16.05      1        35.95         False   
11   149.99          19.77      7       161.42         False   
12   149.99          19.77      7         8.34         False   
13    99.00          30.53      5       259.06         False   
14    99.00          30.53      5       259.06         False   
15    98.00          16.13      7       

In [55]:
from sklearn.model_selection import train_test_split

# Separar las características (X) de la variable objetivo (y)
X = df_model.drop('total_value', axis=1)
y = df_model['total_value']

# Dividir el DataFrame en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.head(10))



     price  freight_value  month  season_Otoño  season_Primavera  \
0    29.99           8.72     10          True             False   
1    29.99           8.72     10          True             False   
2    29.99           8.72     10          True             False   
3   118.70          22.76      7         False             False   
4   159.90          19.22      8         False             False   
5    45.00          27.20     11          True             False   
6    19.90           8.72      2         False             False   
7   147.90          27.36      7         False             False   
9    59.99          15.17      5         False              True   
10   19.90          16.05      1         False             False   

    season_Verano  payment_type_credit_card  payment_type_debit_card  \
0           False                      True                    False   
1           False                     False                    False   
2           False                  

In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
# Estandarizar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y ajustar el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Coeficiente de determinación (R^2): {r2:.2f}")


Error cuadrático medio (MSE): 5243.53
Coeficiente de determinación (R^2): 0.93


El modelo está diseñado para predecir el valor total de una orden (total_value). Este valor incluye el precio de los productos, el valor del envío y otros costos asociados a la compra. Esto permite estimar cual será el valor real que tendrá que pagar un cliente en cada compra. Este conocimiento es importante para la empresa, pues la decisión de compra de un cliente puede variar si el precio inicial de un producto incremeta mucho cuando se incluyen otros costos.



In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Filtrar las columnas relevantes y agrupar por categoría y temporada
df_grouped = df.groupby(['product_category_name', 'season'])['total_value'].sum().reset_index()
df_grouped = pd.get_dummies(df_grouped, columns=['product_category_name', 'season'], drop_first=True)

# Separar características y objetivo
X = df_grouped.drop('total_value', axis=1)
y = df_grouped['total_value']

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y ajustar el modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir resultados
print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Coeficiente de determinación (R^2): {r2:.2f}")

# Análisis de Importancia de Características
importances = model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

print("Importancia de características:")
print(feature_importance_df)



Error cuadrático medio (MSE): 3185221252.64
Coeficiente de determinación (R^2): 0.83
Importancia de características:
                                              Feature  Importance
12              product_category_name_cama_mesa_banho    0.206601
53             product_category_name_moveis_decoracao    0.129379
43       product_category_name_informatica_acessorios    0.114919
31                product_category_name_esporte_lazer    0.092635
65           product_category_name_relogios_presentes    0.083421
..                                                ...         ...
64  product_category_name_portateis_cozinha_e_prep...    0.000020
3            product_category_name_artes_e_artesanato    0.000019
35       product_category_name_fashion_roupa_feminina    0.000018
57                       product_category_name_musica    0.000013
66           product_category_name_seguros_e_servicos    0.000007

[75 rows x 2 columns]


modelo esta diseñado para predecir el valor total de las ventas de una categoria de producto especifica en una temporada del año determinada (varaibles de entrada). Esta información es de especial interes para la empresa pues les permite orgnaizar su inventario y cadenas de proveeduria

Felipe



In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o arquivo Parquet
df = pd.read_parquet('/Users/felipediegolobatodasilva/Desktop/despliegue_analytica/files_parquet/df_joined_cleaned_no_nulls.parquet')

# Transformar a coluna de data em datetime
df['order_purchase_timestamp_dt'] = pd.to_datetime(df['order_purchase_timestamp'])

# Extrair o mês e a temporada
df['month'] = df['order_purchase_timestamp_dt'].dt.month
def assign_season(month):
    if month in [12, 1, 2]:
        return 'Invierno'
    elif month in [3, 4, 5]:
        return 'Primavera'
    elif month in [6, 7, 8]:
        return 'Verano'
    else:
        return 'Otoño'
df['season'] = df['month'].apply(assign_season)

# Verificar e criar a coluna 'total_value' se não estiver presente
if 'total_value' not in df.columns:
    df['total_value'] = df['payment_value']

# Selecionar as colunas relevantes e criar variáveis dummies
variables_relevantes = ['season', 'price', 'freight_value', 'payment_type', 'product_category_name', 'customer_state', 'month', 'total_value']
df_model = df[variables_relevantes].copy()
df_model = pd.get_dummies(df_model, columns=['season', 'payment_type', 'product_category_name', 'customer_state'], drop_first=True)

# Separar características (X) e variável alvo (y)
X = df_model.drop('total_value', axis=1)
y = df_model['total_value']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar as características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar PCA e manter 95% da variância
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Imprimir o número de componentes selecionados
print(f"Número de componentes selecionados: {pca.n_components_}")

# Criar e treinar o modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_pca, y_train)

# Realizar predições
y_pred = model.predict(X_test_pca)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir métricas de avaliação
print(f"Error quadrático médio (MSE): {mse:.2f}")
print(f"Coeficiente de determinação (R^2): {r2:.2f}")


Número de componentes selecionados: 97
Error quadrático médio (MSE): 8676.37
Coeficiente de determinação (R^2): 0.88


In [ ]:
###